In [ ]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
from matplotlib import pyplot as plt
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn import visualize
import paper
import random
from glob import glob

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs

PRETRAINED_MODEL_PATH = "C:/Users/darksoul/Mask_RCNN/mask_rcnn_surgery_0030.h5"
IMAGE_DIR = "C:/Users/darksoul/Mask_RCNN/datasets/4points/val"

In [ ]:
class InferenceConfig(paper.PaperConfig):
            # Set batch size to 1 since we'll be running inference on
            # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
config = InferenceConfig()
config.display()

model = modellib.MaskRCNN(mode="inference", config=config, model_dir='C:/Users/darksoul/Mask_RCNN/logs')
model_path = PRETRAINED_MODEL_PATH
# or if you want to use the latest trained model, you can use : 
# model_path = model.find_last()[1]
model.load_weights(model_path, by_name=True)

# Prediction and merge masks 

In [ ]:
file_names = glob(os.path.join(IMAGE_DIR, "*.jpg"))
masks_prediction = np.zeros((1200, 1600, len(file_names)))
for i in range(len(file_names)):
    print(i)
    image = skimage.io.imread(file_names[i])
    predictions = model.detect([image],  verbose=1)
    p = predictions[0]
    masks = p['masks']
    merged_mask = np.zeros((masks.shape[0], masks.shape[1]))
    for j in range(masks.shape[2]):
        merged_mask[masks[:,:,j]==True] = True
        masks_prediction[:,:,i] = merged_mask
print(masks_prediction.shape)

# Load Annotations

In [ ]:
PREDICT_DIR = '/home/simon/deeplearning/mask_rcnn/data/surgery/'
dataset = paper.PaperDataset()
dataset.load_VIA(PREDICT_DIR, 'predict')

# Calculate Accuracy 

In [ ]:
accuracy = 0
precision = 0
for image_id in range(len(dataset.image_info)):
    name = dataset.image_info[image_id]['id']
    file_name = os.path.join(IMAGE_DIR, name)
    image_id_pred = file_names.index(file_name)
    merged_mask = masks_prediction[:, :, image_id_pred]
    
    annotated_mask = dataset.load_mask(image_id)[0]
    merged_annotated_mask = np.zeros((1200, 1600))
    for i in range(annotated_mask.shape[2]):
        merged_annotated_mask[annotated_mask[:,:,i]==True] = True
    accuracy  += np.sum(merged_mask==merged_annotated_mask) / (1200 * 1600)
    all_correct = np.sum(merged_annotated_mask[merged_mask == 1])
    precision += all_correct / (np.sum(merged_mask))
print('accuracy:{}'.format(accuracy / len(file_names)))
print('precision:{}'.format(precision / len(file_names)))

# Trained with 100 images
Accuracy: 99.49%
Precision:97.02%
# Trained with 200 images
Accuracy: 99.54%
Precision: 96.78%
# Trained with 200 images(60 epochs)
Accuracy: 99.53%
Precision: 97.28%

# Visualize Random IMage

In [ ]:
file_names = glob(os.path.join(IMAGE_DIR, "*.jpg"))
class_names = ['Paper']
test_image = skimage.io.imread(file_names[random.randint(0,len(file_names)-1)])
predictions = model.detect([test_image], verbose=1) # We are replicating the same image to fill up the batch_size
p = predictions[0]
visualize.display_instances(test_image, p['rois'], p['masks'], p['class_ids'], 
                            class_names, p['scores'])